In [1]:
import fitz # PyMuPDF
import pandas as pd
import re
import os
from datetime import datetime

In [2]:
# DIRECTORIES AND FILES

# Directory with the PDFs
root_pdf_dir = r'G:\ACCOUNT PAYABLE\001 - INVOICES - FATTURE\LOGISTIC INVOICES\1- NEW\MSC invoices received from 06-01-2025'

# Directory with the Processed PDFs
processed_dir = r'C:\Users\ACarrion\OneDrive - F.lli De Cecco di Filippo Fara San Martino S.p.A\Documents\Logistics Invoicing\MSC'

# Most recent SOA - NEEDS TO BE CONVERTED FROM 97-2003 Worksheet to regular. The file name will need to be changed to the most recent one
soa_file = r'g:\ACCOUNT PAYABLE\001 - INVOICES - FATTURE\LOGISTIC INVOICES\1- NEW\MSC invoices received from 06-01-2025\SOA for PRODOTTI MEDITERRANEI INC- 8-8-2025_1.xlsx'

# MSC Tracker File
msc_invoice_tracker = r'g:\ACCOUNT PAYABLE\001 - INVOICES - FATTURE\LOGISTIC INVOICES\1- NEW\MSC invoices received from 06-01-2025\MSC Invoice Tracker.xlsx' 

In [3]:
# Recursively collect all PDFs file paths for in the ROOT DIRECTORY
pdf_files = []
for dirpath, dirnames, filenames in os.walk(root_pdf_dir):
    for filename in filenames:
        if filename.lower().endswith('.pdf'):
            pdf_files.append(os.path.join(dirpath, filename))

# Pattern to extract bill of lading
bol_pattern = re.compile(r'\bMEDU\w+\b')
type_keywords = {
    "Rail Detention": 'Rail',
    "IMPORT CHASSIS PER DIEM": "Chassis Per Diem",
    "PER DIEM IMPORT": "Per Diem",
    "FREIGHT INVOICE": "Chassis/Freight"
}

invoice_data_root_directory = []

for filepath in pdf_files:
    with fitz.open(filepath) as doc:
        # Identify the folder where the PDF resides
        folder = os.path.basename(os.path.dirname(filepath))
        # Extract all text for BOL and type classification
        text = ''
        for page in doc:
            text += page.get_text()
        
        # Invoice number = filename without extensions
        filename = os.path.basename(filepath)
        invoice_number = os.path.splitext(filename)[0]

        # Extract first BOL number
        bol_match = bol_pattern.findall(text)
        bol = bol_match[0] if bol_match else ""

        # Classify type from keywords
        invoice_type = "Unknown"
        for keyword, label in type_keywords.items():
            if keyword.lower() in text.lower():
                invoice_type = label
                break
        
        invoice_data_root_directory.append({
            'Folder': folder,
            "Filename": filename,
            "Invoice Number": invoice_number,
            "BOL Number": bol,
            "Invoice Type": invoice_type
        })

# Create dataframe for phase 1 output
df_all_invoices = pd.DataFrame(invoice_data_root_directory)

timestamp  = datetime.now().strftime('%Y%m%d_%H%M')
file_name = f'LIST_ALL_INVOICES_{timestamp}.xlsx'
# Save to MSC INVOICES received from 06-01-2025
df_all_invoices.to_excel(file_name, index=False)

In [4]:
# Recursively collect all PDFs file paths for in the PROCESSED DIRECTORY
pdf_files_processed = []
for dirpath, dirnames, filenames in os.walk(processed_dir):
    for filename in filenames:
        if filename.lower().endswith('.pdf'):
            pdf_files_processed.append(os.path.join(dirpath, filename))

# Pattern to extract bill of lading
bol_pattern = re.compile(r'\bMEDU\w+\b')
type_keywords = {
    "Rail Detention": 'Rail',
    "IMPORT CHASSIS PER DIEM": "Chassis Per Diem",
    "PER DIEM IMPORT": "Per Diem",
    "FREIGHT INVOICE": "Chassis/Freight"
}

invoice_data_processed_directory = []

for filepath in pdf_files_processed:
    with fitz.open(filepath) as doc:
        # Identify the folder where the PDF resides
        folder = os.path.basename(os.path.dirname(filepath))
        # Extract all text for BOL and type classification
        text = ''
        for page in doc:
            text += page.get_text()
        
        # Invoice number = filename without extensions
        filename = os.path.basename(filepath)
        invoice_number = os.path.splitext(filename)[0]

        # Extract first BOL number
        bol_match = bol_pattern.findall(text)
        bol = bol_match[0] if bol_match else ""

        # Classify type from keywords
        invoice_type = "Unknown"
        for keyword, label in type_keywords.items():
            if keyword.lower() in text.lower():
                invoice_type = label
                break
        
        invoice_data_processed_directory.append({
            'Folder': folder,
            "Filename": filename,
            "Invoice Number": invoice_number,
            "BOL Number": bol,
            "Invoice Type": invoice_type
        })

# Create dataframe for phase 1 output
df_processed_invoices = pd.DataFrame(invoice_data_processed_directory)

timestamp  = datetime.now().strftime('%Y%m%d_%H%M')
file_name = f'LIST_ALL_PROCESSED_INVOICES_{timestamp}.xlsx'
# Save to MSC INVOICES received from 06-01-2025
df_processed_invoices.to_excel(file_name, index=False)

In [5]:

# Load sheet Ocean Freight 
df_soa_oceanFreight = pd.read_excel(soa_file, sheet_name='Ocean Freight')
df_soa_oceanFreight.rename(columns={'BOL': 'Invoice Number'}, inplace=True)
df_soa_oceanFreight['BOL'] = df_soa_oceanFreight['Invoice Number']
# Keep Relevant Columns
df_soa_oceanFreight = df_soa_oceanFreight[['Invoice Number', 'BOL', 'Invoice Date', 'Due Date', 'Manifested Amt', 'Paid',
                                           'Outstanding amt#', 'Containers','Disputed?', 'Dispute Status',]]

In [6]:
# Load Non-OFT sheet
df_soa_non_oft = pd.read_excel(soa_file, sheet_name='Non-OFT')
df_soa_non_oft.rename(columns={'BOLs': 'BOL'}, inplace=True)
# Keep Relevant Columns
df_soa_non_oft = df_soa_non_oft[['Invoice Number', 'Invoice Date', 'Manifested Amt', 'Paid', 'Outstanding amt#', 'BOL', 'Containers',
                                 'Disputed?',  'Dispute Status']]


In [7]:
# Combine both sheets
soa_combined = pd.concat([df_soa_oceanFreight, df_soa_non_oft], axis=0, join='outer')

In [8]:
# Phase 3: Read multiple Bruzzone Files (YTD and last year)
bruzzone_files = [
    r'c:\Users\ACarrion\OneDrive - F.lli De Cecco di Filippo Fara San Martino S.p.A\Documents\Logistics Invoicing\Bruzzone\2025-YTD-Consolidated-Invoices.xlsx',
    r'c:\Users\ACarrion\OneDrive - F.lli De Cecco di Filippo Fara San Martino S.p.A\Documents\Logistics Invoicing\Bruzzone\2024\2024-Consolidated-Invoices.xlsx'
]

bruzzone_dfs = []
needed_cols = ['BILL OF LADING #', 'DELIVER TO NAME', 'Customer Ref # 1']
for bf in bruzzone_files:
    df_temp = pd.read_excel(bf)
    # Keep only the needed columns if they exist
    available = [c for c in needed_cols if c in df_temp.columns]
    df_temp = df_temp[available]
    # Rename to uniform names
    df_temp.rename(columns={
        'BILL OF LADING #': 'BOL',
        'DELIVER TO NAME': 'Customer from Bruzzone File',
        'Customer Ref # 1': 'PO'
    }, inplace=True)

    # Ensure all three columns exist, fill missing
    for col in ['BOL', 'PO', 'Customer']:
        if col not in df_temp.columns:
            df_temp[col] = pd.NA
    df_temp = df_temp[['BOL', 'PO', 'Customer from Bruzzone File']]
    bruzzone_dfs.append(df_temp)

df_bruzzone = pd.concat(bruzzone_dfs, ignore_index=True)
df_bruzzone.drop_duplicates(subset=['BOL'], inplace=True)

In [9]:
df_customer = pd.merge(soa_combined, df_bruzzone, on='BOL', how='left')

In [10]:
pipeline_file = ''
transportation_file = ''
for file in os.listdir():
    if file.startswith('SAP_Pipeline'):
        pipeline_file = file
    if file.startswith('SAP_Transportation'):
        transportation_file = file

print(pipeline_file)
print(transportation_file)

SAP_Pipeline_8.4.xlsx
SAP_Transportation_8.4.xlsx


In [11]:
# Merge with pipeline file
df_pipe = pd.read_excel(pipeline_file)
df_pipe = df_pipe[['Purchasing Document', 'Sales Document', 'Sold-to Name']]
df_pipe.rename(columns={'Purchasing Document': 'PO', 'Sold-to Name': 'Customer Name in SAP'}, inplace=True)
# Fill missing sales with PO
df_pipe['Sales Document'] = df_pipe['Sales Document'].fillna(df_pipe['PO'])
# Merge onto customer
df_customer = pd.merge(df_customer, df_pipe, on='PO', how='left')

c:\conda\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:81: UserWarning: Cell V2015 is marked as a date but the serial value -693595 is outside the limits for dates. The cell will be treated as an error.
  for idx, row in parser.parse():


In [12]:
# Merge with Transportation file
df_transportation = pd.read_excel(transportation_file)
df_transportation = df_transportation[['Sales Document', 'Service agent description']]
df_transportation.rename(columns={'Service agent description': 'Carrier'}, inplace=True)
# Merge onto customer
df_customer = pd.merge(df_customer, df_transportation, on='Sales Document', how='left')
df_customer

,Invoice Number,BOL,Invoice Date,Due Date,Manifested Amt,Paid,Outstanding amt#,Containers,Disputed?,Dispute Status,PO,Customer from Bruzzone File,Sales Document,Customer Name in SAP,Carrier
0,MEDUGD857751,MEDUGD857751,2025-01-17 00:00:00,2025-01-17,350.0,0.0,350.0,CAAU5579511,False,NaN,4500199830,DAIRYLAND,2.971580e+05,DAIRYLAND USA CORP. East,NaN
1,MEDUGD845764,MEDUGD845764,2025-01-20 00:00:00,2025-01-20,700.0,0.0,700.0,MSNU7467194; MSNU9627178,False,NaN,4500201306,HUB GROUP CASESTACK,4.500201e+09,PMI - Casestack Plant,NaN
2,MEDUGD915278,MEDUGD915278,2025-01-22 00:00:00,2025-01-22,350.0,0.0,350.0,CAAU7810430; GAOU7373877,False,NaN,4500201383,PRODOTTI MEDITERRANEI INC,2.994730e+05,Restaurant Depot Enterprices,NaN
3,MEDUGD970489,MEDUGD970489,2025-01-29 00:00:00,2025-01-29,700.0,0.0,700.0,BEAU5709125; TRHU5118612,False,NaN,4500201439,EUROPEAN IMP C/O CENTRAL WHSE,2.994860e+05,EUROPEAN IMPORTS LTD,Contra America Corp
4,MEDUGD845368,MEDUGD845368,2025-02-06 00:00:00,2025-02-06,350.0,0.0,350.0,MSNU7887432; TIIU4840631,False,NaN,4500201226,UNFI RICHURG,2.987670e+05,"UNITED NATURAL FOODS, INC. / N",Contra America Corp
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,100003327176P,MEDUGK049005,2025-07-30 00:00:00,NaT,250.0,0.0,250.0,MSMU7643685,False,NaN,4500213883,PRISM TEAM SERVICES INC,4.500214e+09,PRISM,NaN
142,100003330924P,MEDUGK048908,2025-08-04 00:00:00,NaT,750.0,0.0,750.0,MSNU7716234,False,NaN,4500213418,PRISM TEAM SERVICES INC,4.500213e+09,PRISM,NaN
143,100003330925P,MEDUGK138535,2025-08-04 00:00:00,NaT,250.0,0.0,250.0,MSCU5192582,False,NaN,4500214127,KEHE FOOD DISTRIBUTORS INC,3.150410e+05,KEHE DISTRIBUTORS LLC.,Contra America Corp
144,100000696441Z,MEDUGK195964,2025-08-07 00:00:00,NaT,50.0,0.0,50.0,MSMU7235488,False,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# Check if we have the pdfs for the invoices in the statement
minimal_invoices = set(df_all_invoices['Invoice Number'])
df_customer['Invoice on File'] = df_customer['Invoice Number'].apply(lambda x: 'Yes' if x in minimal_invoices else 'No')

In [14]:
# Check if invoices in the folder are NOT in the statement
invoices_check = pd.merge(df_all_invoices, df_customer[['Invoice Number', 'BOL']], on='Invoice Number', how='left', indicator=True)
invoices_check.to_excel('Invoices_Check.xlsx', index=False)

In [15]:
df_invoice_tracker = pd.read_excel(msc_invoice_tracker, sheet_name= 'MSC Open Invoices')
# df_invoice_tracker.columns
df_invoice_tracker = pd.merge(df_invoice_tracker, df_bruzzone[['BOL', 'PO']], on='BOL', how='left')
df_invoice_tracker = pd.merge(df_invoice_tracker, df_pipe[['PO', 'Customer Name in SAP', 'Sales Document']], on='PO', how='left')
df_invoice_tracker = pd.merge(df_invoice_tracker, df_transportation[['Sales Document', 'Carrier']], on='Sales Document', how='left' )
df_invoice_tracker

,Invoice Date,Invoice Number,BOL,Container Number,Service,Amount,Invoice Status,Notes,PO,Customer Name in SAP,Sales Document,Carrier
0,2025-01-20,MEDUGD845764,MEDUGD845764,MSNU7467194; MSNU9627178,Chassis Usage Charge,700.0,Reviewing,Chassis was paid to Cargomatic. Sent them an e...,4500201306,PMI - Casestack Plant,4.500201e+09,NaN
1,2025-03-08,MEDUFK175133,MEDUFK175133,TXGU7209120,Chassis Usage Charge,350.0,Reviewing,emailed DCS to review on 7/28,4500204828,PMI - Casestack Plant,4.500205e+09,NaN
2,2025-03-20,MEDUFK195636,MEDUFK195636,MSDU8455583,Chassis Usage Charge,350.0,Reviewing,emailed DCS to review on 7/28,4500204827,PMI - Casestack Plant,4.500205e+09,NaN
3,2025-03-27,MEDUFK343764,MEDUFK343764,MSBU8162994; MSDU6992834,Chassis Usage Charge,700.0,Reviewing,emailed DCS to review on 7/28,4500206240,PMI - Casestack Plant,4.500206e+09,NaN
4,2025-04-07,MEDUFK376723,MEDUFK376723,MSMU4739720; UETU7052570,Chassis Usage Charge,700.0,Reviewing,emailed DCS to review on 7/28,4500206511,PMI - Casestack Plant,4.500207e+09,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
116,2025-06-02,100003291575P,MEDUGK379790,MSNU6707829,PER DIEM IMPORT,250.0,Processed,NaN,NaN,NaN,NaN,NaN
117,2025-04-21,100003240614P,MEDUFK363879,MSBU8172610,PER DIEM IMPORT,185.0,Disputing,Per diem starts at 6 days?,4500205588,Lipari Foods Operating Company,3.048060e+05,Contra America Corp
118,2025-04-21,100003240613P,MEDUFK363879,MSMU4223673,PER DIEM IMPORT,185.0,Disputing,Per diem starts at 6 days?,4500205588,Lipari Foods Operating Company,3.048060e+05,Contra America Corp
119,2025-04-09,100003232004P,MEDUFK371138,MSBU5061418,PER DIEM IMPORT,185.0,Disputing,Per diem starts at 6 days?,4500206022,Woodland Partners Inc. SDS,3.037630e+05,Del Corona & Scardigli


In [16]:
processed_status = 'Processed'

def _norm_series(s):
    return s.fillna("").astype(str).str.strip().str.upper()
batch_ts = datetime.now().strftime('%Y-%m-%d %H:%M')


df_invoice_tracker.columns

Index(['Invoice Date', 'Invoice Number', 'BOL', 'Container Number', 'Service',
       'Amount ', 'Invoice Status', 'Notes', 'PO', 'Customer Name in SAP',
       'Sales Document', 'Carrier'],
      dtype='object')

In [17]:
# PDFs on file but not in SOA
soa_inv_set = set(_norm_series(soa_combined['Invoice Number'])) if 'Invoice Number' in soa_combined.columns else set()

min_inv_norm = _norm_series(df_all_invoices['Invoice Number'])
mask_not_in_soa = ~min_inv_norm.isin(soa_inv_set)
df_candidates = df_all_invoices[mask_not_in_soa].copy()

# Exclude anything already present in the tracker (by Invoice Number only)
trk_inv_set = set(_norm_series(df_invoice_tracker.get('Invoice Number', pd.Series([]))))
cand_inv_norm = _norm_series(df_candidates['Invoice Number'])
mask_not_in_trk = ~cand_inv_norm.isin(trk_inv_set)
df_to_add_not_sao = df_candidates[mask_not_in_trk].copy()

In [18]:
# Enrich new rows with system facts (PO/Customer/Carrier) from df_customer(by Invoice number only)
sys_cols = ['Invoice Number', 'BOL', 'PO', 'Customer Name in SAP', 'Carrier', 'Invoice Type']
sys_facts = df_customer.copy()
for c in sys_cols:
    if c not in sys_facts.columns:
        sys_facts[c] = pd.NA
sys_facts = sys_facts[sys_cols].drop_duplicates(subset=['Invoice Number'])

add_1 = pd.merge(df_to_add_not_sao, sys_facts, on='Invoice Number', how='left')

new_rows_not_soa = pd.DataFrame({
    'Invoice Date': pd.NaT,
    'Invoice Number': add_1['Invoice Number'],
    'BOL': add_1['BOL'].fillna(add_1['BOL Number']),
    'Container Number': pd.NA,
    'Amount': pd.NA,
    'Invoice Status': pd.NA,
    'Notes': 'Not on recent SOA ' + f' batch {batch_ts}',
    'Customer': add_1.get('Customer Name in SAP', pd.NA),
    'PO': add_1.get('PO', pd.NA),
    'Carrier' : add_1.get('Carrier', pd.NA),
})

In [19]:
# Mark "Processed from a folder"
proc_records = []
for dirpath, _, filenames in os.walk(processed_dir):
    for fn in filenames:
        if fn.lower().endswith('.pdf'):
            fp = os.path.join(dirpath, fn)
            inv_num = os.path.splitext(fn)[0] # Invoice number is in the filename (no extension)
            # Try to read BOL + Service for completeness
            bol_val, invoice_type = "", "Unknown"
            try:
                with fitz.open(fp) as doc:
                    text = ''.join([p.get_text() for p in doc])
                bol_match = re.findall(r'\bMEDU\w+\b', text)
                bol_val = bol_match[0] if bol_match else ""
                for keyword, label in type_keywords.items():
                    invoice_type = label
                    break
            except Exception:
                pass

            proc_records.append({
                'Invoice Number': inv_num,
                'BOL': bol_val,  
            })
df_processed = pd.DataFrame(proc_records) if proc_records else pd.DataFrame(columns=['Invoice Number', 'BOL', 'Service'])


In [20]:
# Update exisiting tracker rows (invoice-only) or add new processed rows
inv_norm_tracker = _norm_series(df_invoice_tracker['Invoice Number'])
updates = 0
adds_processed = pd.DataFrame(columns=df_invoice_tracker.columns)

if not df_processed.empty:
    for _, row in df_processed.iterrows():
        inv = str(row['Invoice Number']).strip()
        hit = (inv_norm_tracker == inv.upper())

        if hit.any():
            idx = hit[hit].index[0]
            # Set status to processed; append not tag once
            df_invoice_tracker.at[idx, 'Invoice Status'] = processed_status
            prev_note = (df_invoice_tracker.at[idx, 'Notes'] or "")
            if f'Processed (batch {batch_ts})' not in str(prev_note):
                sep = ' | ' if prev_note else ''
                df_invoice_tracker.at[idx, 'Notes'] = f'{prev_note}{sep} Processed (batch {batch_ts})'
            # Fill blanks only
            if not df_invoice_tracker.at[idx, 'BOL'] and row['BOL']:
                df_invoice_tracker.at[idx,'BOL'] = row['BOL']
            if (not df_invoice_tracker.at[idx, 'Service']) and pd.notna(row['Service']):
                df_invoice_tracker.at[idx, 'Service'] = row['Service']
            updates += 1
else:
    # Append new Processed row if invoice not in tracker yet
    adds_processed = pd.concat([
        adds_processed,
        pd.DataFrame([{
            'Invoice Date': pd.NaT,
            'Invoice Number': inv,
            'BOL': row['BOL'] if row['BOL'] else pd.NA,
            'Container Number': pd.NA,
            'Service': row['Service'],
            'Amount': pd.NA,
            'Invoice Status': processed_status,
            'Notes': f'Processed (batch {batch_ts})',
            'Customer': pd.NA,
            'PO': pd.NA,
            'Carrier': pd.NA,

        }])
    ], ignore_index=True)

In [21]:
# Write tracker output
df_tracker_out = df_invoice_tracker.copy()
if len(new_rows_not_soa):
    # Align to tracker columns
    for col in df_tracker_out.columns:
        if col not in new_rows_not_soa.columns:
            new_rows_not_soa[col] = pd.NA
    new_rows_not_soa = new_rows_not_soa[df_tracker_out.columns]
    df_tracker_out = pd.concat([df_tracker_out, new_rows_not_soa], ignore_index=True)

if len(adds_processed):
    # Align to tracker columns
    for col in df_tracker_out.columns:
        if col not in adds_processed.columns:
            adds_processed[col] = pd.NA
    adds_processed = adds_processed[df_tracker_out.columns]
    df_tracker_out = pd.concat([df_tracker_out, adds_processed], ignore_index=True)

tracker_out = f"MSC Invoice Tracker - Updated_{timestamp}.xlsx"
with pd.ExcelWriter(tracker_out, engine='openpyxl') as xw:
    df_tracker_out.to_excel(xw, sheet_name='MSC Invoices', index=False)

print(f"Tracker updates: status-updated={updates}, not-on-SOA-added={len(new_rows_not_soa)}, processed-added={len(adds_processed)}")
print(f"Wrote: {tracker_out}")

Tracker updates: status-updated=9, not-on-SOA-added=3, processed-added=0
Wrote: MSC Invoice Tracker - Updated_20250811_1608.xlsx


C:\Users\ACarrion\AppData\Local\Temp\ipykernel_4584\361023155.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_tracker_out = pd.concat([df_tracker_out, new_rows_not_soa], ignore_index=True)


In [22]:
tracker_notes = (df_tracker_out[['Invoice Number', 'Invoice Status', 'Notes']].astype({'Invoice Number': 'string'}).drop_duplicates(subset=['Invoice Number'], keep='first'))

final_report = (
    df_customer.astype({'Invoice Number': 'string'}).merge(tracker_notes, on='Invoice Number', how='left')
)

final_report.to_excel(f'SOA_CHECK{timestamp}.xlsx', index=False)

